In [2]:
import tensorflow as tf
tf.__version__

'2.1.0'

In [3]:
from tensorflow.python.client import device_lib
device_lib.list_local_devices()

[name: "/device:CPU:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 2532688127768558292]

In [4]:
import platform
platform.platform()

'Windows-10-10.0.18362-SP0'

In [1]:
import os

dev_wav_path = "C:\\Users\\오명교\\Desktop\\dev\\wav"
test_wav_path = "C:\\Users\\오명교\\Desktop\\test\\wav"

print(sorted(os.listdir(dev_wav_path))[:5])
print(len(sorted(os.listdir(dev_wav_path)))) # # of speakers

['id10001', 'id10002', 'id10003', 'id10004', 'id10005']
1211


In [2]:
num_dev_labels = os.listdir(dev_wav_path)
num_dev_utts = 0
for dir_path, dir_names, filenames in os.walk(dev_wav_path):
    num_dev_utts += len(filenames)

num_test_labels = os.listdir(test_wav_path)
num_test_utts = 0
for dir_path, dir_names, filenames in os.walk(test_wav_path):
    num_test_utts += len(filenames)

print("[ dev] {:>5} speakers, {:>7} utterances".format(len(num_dev_labels), num_dev_utts))
print("[test] {:>5} speakers, {:>7} utterances".format(len(num_test_labels), num_test_utts))

[ dev]  1211 speakers,  148642 utterances
[test]    40 speakers,    4874 utterances


In [3]:
import os.path

dev_npz_path = os.path.join(os.path.dirname(dev_wav_path), "npz")
test_npz_path = os.path.join(os.path.dirname(test_wav_path), "npz")

if not os.path.exists(dev_npz_path): os.mkdir(dev_npz_path)
if not os.path.exists(test_npz_path): os.mkdir(test_npz_path)

print("[ dev] path: {}".format(dev_npz_path))
print("[test] path: {}".format(test_npz_path))

[ dev] path: C:\Users\오명교\Desktop\dev\npz
[test] path: C:\Users\오명교\Desktop\test\npz


In [4]:
from scipy.io import wavfile
import numpy as np

def wav_to_npz(wav_path, npz_path, data_length = 3 ** 9):
    # change wav folder path to npz folder path
    change_path = lambda x: x.replace(os.path.sep + 'wav' + os.path.sep,
                                      os.path.sep + 'npz' + os.path.sep)
    
    for dir_path, dir_names, filenames in os.walk(wav_path):
        # e.g. dir_path == /dev/wav/id10001 → make 1211 (# of speakers) folders
        if dir_path is wav_path:
            for dir_name in dir_names: # e.g. id10001, id10002, ...
                path = change_path(os.path.join(dir_path, dir_name))
                if not os.path.exists(path): os.mkdir(path)
                
        # e.g. /dev/wav/id10001
        if not filenames: continue
        
        # e.g. /dev/wav/id10001/1zcIwhmdeo4 → /dev/npz/id10001/1zcIwhmdeo4
        path = change_path(dir_path)
        data = []
        
        for filename in filenames:
            _, samples = wavfile.read(os.path.join(dir_path, filename))
            
            # if wave file samples < data length == 3^9, 
            # then raise exception
            try: 
                if len(samples) < data_length:
                    raise Exception("length of samples < data_length = 3 ** 9")
            except Exception as e:
                print(e)

            start = int(len(samples) - data_length) // 2
            end = start + data_length
            
            data.append(samples[start:end])
            
        # e.g. save every files in /dev/wav/id10001/1zcIwhmdeo4 to /dev/npz/id10001/1zcIwhmdeo4.npz
        np.savez_compressed(path, names = filenames, data = data)

In [5]:
data_length = 3 ** 9

In [6]:
wav_to_npz(dev_wav_path, dev_npz_path)

In [7]:
wav_to_npz(test_wav_path, test_npz_path)

In [10]:
def utts(path):
    num_utts = 0
    for dir_path, dir_names, filenames in os.walk(path):
        for filename in filenames:
            num_utts += len(np.load(os.path.join(dir_path, filename))['names'])
    return num_utts

print("[ dev] {:>7} utterances".format(utts(dev_npz_path)))
print("[test] {:>7} utterances".format(utts(test_npz_path)))

[ dev]  148642 utterances
[test]    4874 utterances
